# 강남구 주정차 위반 단속 데이터 분석

이 노트북은 서울시 강남구의 주정차 단속 데이터를 분석하여 시간대별 및 위치별 단속 현황을 파악합니다.

## 1. 데이터 로드 및 전처리

In [14]:
import pandas as pd
import os

file_name = '서울특별시 강남구_주정차위반단속위치현황_20240320.csv'

# 파일이 존재하는지 확인
if not os.path.exists(file_name):
    print(f"오류: '{file_name}' 파일을 찾을 수 없습니다.")
else:
    print("파일을 로드합니다...")
    try:
        # 한글 데이터는 주로 cp949 또는 euc-kr 인코딩 사용
        df = pd.read_csv(file_name, encoding='cp949')
        print("데이터 로드 완료!")
        display(df.head())
    except UnicodeDecodeError:
        print("cp949 디코딩 실패, euc-kr로 시도합니다.")
        df = pd.read_csv(file_name, encoding='euc-kr')
    except Exception as e:
        print(f"파일 로드 중 오류 발생: {e}")

파일을 로드합니다...
데이터 로드 완료!


,집계년도,시군명,시군코드,관리기관명,단속일시,단속동,단속장소,단속구분
0,2021,서울특별시,3245232,강남구,2021-10-01 00:00,역삼동,테헤란로 237 주변,CCTV
1,2021,서울특별시,3245150,강남구,2021-10-01 00:09,역삼동,테헤란로 237 주변,CCTV
2,2021,서울특별시,3252459,강남구,2021-10-01 00:10,삼성동,172-90,안전신문고(앱)
3,2021,서울특별시,3252485,강남구,2021-10-01 00:10,삼성동,172-90,안전신문고(앱)
4,2021,서울특별시,3252484,강남구,2021-10-01 00:12,삼성동,172-90,안전신문고(앱)


## 2. 데이터 전처리

- 필수 컬럼(`단속일시`, `단속동`, `단속장소`)의 결측치를 제거합니다.
- `단속일시`를 날짜 형식으로 변환하고 오류 데이터는 제거합니다.
- 시간을 기준으로 4개의 시간대(야간, 오전, 오후, 저녁)로 분류합니다.
- `단속동`과 `단속장소`를 합쳐 `단속위치` 컬럼을 생성합니다.

In [15]:
# 필수 컬럼 정의
required_columns = ['단속일시', '단속동', '단속장소']

# 결측치 제거
df = df.dropna(subset=required_columns)

# 날짜 변환 (오류 데이터는 NaT로 변환)
df['단속일시'] = pd.to_datetime(df['단속일시'], errors='coerce')

# 변환 실패 데이터(NaT) 제거 및 개수 확인
nat_count = df['단속일시'].isna().sum()
if nat_count > 0:
    print(f"{nat_count}개의 유효하지 않은 날짜 데이터를 제거했습니다.")
    df = df.dropna(subset=['단속일시'])

# 시간대 분류 함수
def get_time_period(hour):
    if 0 <= hour < 6:
        return '0시~6시(야간)'
    elif 6 <= hour < 12:
        return '6시~12시(오전)'
    elif 12 <= hour < 18:
        return '12시~18시(오후)'
    else: # 18 <= hour < 24
        return '18시~24시(저녁)'

# 시간대 컬럼 생성
df['시간대'] = df['단속일시'].dt.hour.apply(get_time_period)

# 단속위치 생성 (단속동 + 단속장소)
df['단속위치'] = df['단속동'].astype(str) + " " + df['단속장소'].astype(str)

print("전처리 완료!")
display(df[['단속일시', '시간대', '단속위치']].head())

1개의 유효하지 않은 날짜 데이터를 제거했습니다.
전처리 완료!


,단속일시,시간대,단속위치
0,2021-10-01 00:00:00,0시~6시(야간),역삼동 테헤란로 237 주변
1,2021-10-01 00:09:00,0시~6시(야간),역삼동 테헤란로 237 주변
2,2021-10-01 00:10:00,0시~6시(야간),삼성동 172-90
3,2021-10-01 00:10:00,0시~6시(야간),삼성동 172-90
4,2021-10-01 00:12:00,0시~6시(야간),삼성동 172-90


## 3. 데이터프레임 생성 및 분석

- 행: `단속위치`
- 열: `시간대`
- 값: 단속 건수

In [16]:
# 피벗 테이블 생성
result_df = df.pivot_table(index='단속위치', columns='시간대', aggfunc='size', fill_value=0)

# 컬럼 순서 정렬
col_order = ['0시~6시(야간)', '6시~12시(오전)', '12시~18시(오후)', '18시~24시(저녁)']
existing_cols = [c for c in col_order if c in result_df.columns]
result_df = result_df[existing_cols]

print("분석 결과 데이터프레임 (상위 5건):")
display(result_df.head())

분석 결과 데이터프레임 (상위 5건):


시간대,0시~6시(야간),6시~12시(오전),12시~18시(오후),18시~24시(저녁)
단속위치,,,,
가락동 서울 송파구 삼학사로2길 49 (가락동),0,0,1,0
강남구 강남구 강남대로 112길,0,0,1,0
강남구 경기도 수원시 권선구 곡반정동 578,1,0,0,0
강남구 서울 강남구 역삼동 801-35,0,0,1,0
강남구 서울시특별시 강남구 역삼동 825-17,0,1,0,0


## 4. 최종 결과 도출

- 가장 단속이 많이 일어난 시간대
- 가장 단속이 많이 일어난 구역
- **(추가) 특정 시간대와 구역의 조합 중 가장 많은 단속이 일어난 경우**

In [17]:
# 1. 가장 단속이 많이 일어난 시간대
time_counts = result_df.sum(axis=0)
most_freq_time = time_counts.idxmax()
most_freq_time_count = time_counts.max()

print(f"1. 가장 단속이 많이 일어난 시간대: {most_freq_time} ({most_freq_time_count:,}건)")

# 2. 가장 단속이 많이 일어난 구역
location_counts = result_df.sum(axis=1)
most_freq_loc = location_counts.idxmax()
most_freq_loc_count = location_counts.max()

print(f"2. 가장 단속이 많이 일어난 구역: {most_freq_loc} ({most_freq_loc_count:,}건)")

# 3. 특정 시간대와 구역의 조합 중 가장 많은 단속이 일어난 경우
# stack()을 사용하여 데이터를 1차원으로 만든 후 가장 큰 값을 찾습니다.
max_case = result_df.stack().idxmax()
max_count = result_df.stack().max()

print(f"3. 가장 단속이 많이 일어난 특정 시간대와 구역: {max_case[1]}에 {max_case[0]} ({max_count:,}건)")

1. 가장 단속이 많이 일어난 시간대: 12시~18시(오후) (257,600건)
2. 가장 단속이 많이 일어난 구역: 압구정동 압구정로29길 72-1 주변 (5,875건)
3. 가장 단속이 많이 일어난 특정 시간대와 구역: 12시~18시(오후)에 압구정동 압구정로29길 72-1 주변 (3,318건)
